In [ ]:
import pandas as pd
import numpy as np
import itertools as it
import random as rd
import networkx as nx
import re
debug=False

In [ ]:
%matplotlib

In [ ]:
def probElements(elements):
    dictResult={}
    sumFreq=sum([v for k,v in elements.iteritems()])
    for k,v in elements.iteritems():
        dictResult[k]=float(v)/sumFreq
    return dictResult
    
def distElements(elements):
    prop=[float(1)/c for c in range(1,len(elements)+1)]
    sumProp=sum(prop)
    dist=[p/sumProp for p in prop]
    return dist

## Paramétrage du paradigme
Les marks définissent les cases du paradigme
- le nom de la case correspond à la concaténation des valeurs possibles
 - abc correspond à une case qui peut prendre comme valeur a, b ou c
 
Les classes sont calculées en faisant le produit cartésien des différentes valeurs x cases

In [ ]:
marks=["abc","def","ghi","klm","nop","qrs","tuv","wxy"]
marksN=[m+"N" for m in marks]
distMarks=distElements(marksN)

classes=pd.DataFrame(columns=marks)

cf=0
for element in it.product(*marks):
    cf+=1
    row = pd.Series({m:element[n] for n,m in enumerate(marks)},name=cf)
    classes=classes.append(row)

nbClasses=len(classes)
classes = classes.sample(frac=1).reset_index(drop=True)
#classes

In [ ]:
nbGrammaireCF=len(classes)

## Paramètres du lexique
On fixe le nombre de lexèmes présents dans le corpus d'apprentissage et le ratio de formes attestées
- nombre de lexèmes
- ratio des formes-cases attestées par rapport au potentiel complet
 - 25% correspond au ratio observé pour les verbes dans Lex3
- nbFormes est l'ordre de grandeur à respecter pour l'échantillon en nombre de types

In [ ]:
ratio=1.0
nbLexemes=25000

nbFormes=int(nbLexemes*len(marks)*ratio)
nbFormes

## Distribution des lexèmes et des CF
Les différents éléments suivent des distributions par Zipf.
- distElements renvoie une liste de probabilités correspondant au nombre d'éléments suivant une Zipf(x)

Chaque lexème a une fréquence de lemme qui correspond à son rang et reçoit une CF tirée au hasard suivant une loi de Zipf.
- np.random.choice(cfs,p=distCFs)

In [ ]:
cfs=range(nbClasses)
distCFs=distElements(cfs)

lexs=range(nbLexemes)
distLexs=distElements(lexs)

lexemes=pd.DataFrame(columns=["CF","P"]+marks)
for l in range(nbLexemes):
    cf=np.random.choice(cfs,p=distCFs)
    dictL={"CF":int(cf), "P":distLexs[l]}
    dictL.update(classes.iloc[cf].to_dict())
    row=pd.Series(dictL,name=l)
    lexemes=lexemes.append(row)
#lexemes.index+=1

In [ ]:
lexemes

In [ ]:
print "Nombre de types potentiel",lexemes[marks].count().sum()

## Nombre de CF tirées dans le lexique
Ce nombre représente le maximum de CF qui pourraient être nécessaires pour la description.
Le nombre de CF nécessaire est au moins égal au nombre de CF qui possèdent un paradigme exemplaire, mais les CF qui n'ont qu'une représentation partielle de leur paradigme ne sont pas obligatoirement nécessaires à la description.

In [ ]:
nbLexiqueCF=len(lexemes.groupby("CF").groups.keys())
print nbLexiqueCF,"CF dans le lexique sur",nbGrammaireCF,"CF dans la grammaire"

## Constitution du DF pour le tirage

In [ ]:
tiragesColonnes=pd.DataFrame(0, index=np.arange(len(lexs)), columns=marksN)
tirages=pd.concat([lexemes, tiragesColonnes], axis=1, sort=False)
#tirages

### Tirage des lexèmes pour les formes attestées
- lexTirs contient le nombre de token à tirer pour chaque lexème

=> changer le random.choice pour faire le choix en une seule fois et ensuite calculer les tirs pour chaque lexème

In [ ]:
#tirages=pd.DataFrame(0, index=np.arange(len(lexs)), columns=marks)
#tirages.index+=1

nbTokens=5000000
lexTirs={}
listeTirs=np.random.choice(lexs,nbTokens,replace=True,p=distLexs)
for t in listeTirs:
    if not t in lexTirs:
        lexTirs[t]=0
    lexTirs[t]+=1
#lexTirs

In [ ]:
newLexFreq=probElements(lexTirs)

In [ ]:
for l in newLexFreq:
    print l,"%0.2f"%((newLexFreq[l]-distLexs[l])/distLexs[l]*100)

### Tirage des formes-cases
- cellTirs contient le nombre de fois que chaque forme-case a été tirée

In [ ]:
for l in lexTirs:
    cellTirs={}
    listeCases=np.random.choice(marksN,lexTirs[l],replace=True,p=distMarks)
    for c in listeCases:
        if not c in cellTirs:
            cellTirs[c]=0
        cellTirs[c]+=1
    if len(cellTirs)==8 and debug:
        print l,tirages.loc[l,"CF"], cellTirs
    for c in cellTirs:
        tirages.loc[l,c]=cellTirs[c]
    

### Nombre de formes du tirage brut
Le nombre de formes du tirage brut correspond au nombre de tokens paramétré si ce nombre est plus élevé que le nombre de formes calculé via le *ratio*, le tirage est réduit pour obtenir un nombre de forme de l'ordre de grandeur désiré

In [ ]:
tirages[marksN].replace(0,np.nan).count().sum()

## Nouvelles fréquences des cases

In [ ]:
newDistMarks=probElements(tirages[marksN].sum().to_dict())
distMarks,newDistMarks

## Réduction du nombre de types
Pour obtenir un nombre de type compatible avec l'ordre de grandeur fixé via *ratio*, on fixe un seuil de tokens pour inclure les formes dans le tirage.
- si le seuil est fixé à 3, par exemple, les formes ayant moins de 3 attestations sont éliminées
- le seuil est calculé pour s'approcher de l'ordre de grandeur par le haut

Les lexèmes qui n'ont aucune forme dans l'échantillon sont éliminés.
- result=result.dropna(thresh=len(marks)+2+1) => un lexème qui a au moins une forme doit avoir ses 2 colonnes CF, P remplies ainsi que toutes les marques, len(marks), plus au moins une forme tirée (+1)

In [ ]:
def reduceTirages(df,seuil):
    result=df.copy()
    for n in range(seuil+1):
        result[marksN]=result[marksN].replace(n,np.nan)
    result=result.dropna(thresh=len(marks)+2+1)
    return result

In [ ]:
tiragesReduits=tirages.copy()
for i in range(nbFormes):
    if reduceTirages(tirages,i)[marksN].count().sum()<nbFormes:
        break
if i>0:
    tiragesReduits=reduceTirages(tirages,i-1)
print "Nombre de types réduit pris en compte",tiragesReduits[marksN].count().sum()

In [ ]:
#tiragesReduits

In [ ]:
print "Nombre de lexèmes dans l'échantillon",len(tiragesReduits)

## Ajouter un champ pour regex

In [ ]:
def ajouterChampParadigme(x):
    result=""
    for c in marks:
        if x[c+"N"]>0 and x[c]==x[c]:
            result+=x[c]
        else:
            result+="."
    return result

In [ ]:
tiragesReduits["regex"]=tiragesReduits.apply(ajouterChampParadigme,axis=1)

In [ ]:
tiragesReduits[marksN]=tiragesReduits[marksN].replace({0:np.nan})

In [ ]:
fullParadigms=tiragesReduits.dropna()
print
print "Nombre de lexèmes avec un paradigme complet dans l'échantillon",len(fullParadigms)
fullParadigms

In [ ]:
nbCompleteCF=len(fullParadigms.groupby("regex"))
print "Nombre de CF exemplaires pleines dans l'échantillon",nbCompleteCF

In [ ]:
#fullParadigms.groupby("regex").count()[["P"]].sort_values("P")

In [ ]:
paradigmsCounts=tiragesReduits.groupby(["regex"]).count()
print len(paradigmsCounts)
paradigmsCounts=paradigmsCounts["P"].sort_values().to_dict()

In [ ]:
paradigmsGroups=tiragesReduits.groupby(["regex"]).groups.keys()
print len(paradigmsGroups)

In [ ]:
paradigmMappings={}
for p in paradigmsGroups:
    for cfRegex in paradigmsGroups:
        m=re.match(p,cfRegex)
        if m:
            if not p in paradigmMappings:
                paradigmMappings[p]=set()
            paradigmMappings[p].add(cfRegex)
paradigmMappings

## Trouver les paradigmes restants

In [ ]:
paradigmTops={}
for p in paradigmMappings:
    if len(paradigmMappings[p])==1:
        paradigmTops[p]=paradigmsCounts[p]
len(paradigmTops),paradigmTops

## Trouver les correspondances des partiels vers les restants

=> mettre à jour le lexique avec les nouvelles classes flexionnelles

In [ ]:
import operator
paradigm2Top={}
top2Paradigms={}
for p in paradigmMappings:
    lTops={m:paradigmTops[m] for m in paradigmMappings[p] if m in paradigmTops}
    topMax=max(lTops.iteritems(), key=operator.itemgetter(1))[0]
    if topMax not in top2Paradigms:
        top2Paradigms[topMax]=[]
    top2Paradigms[topMax].append((p,paradigmsCounts[p]))
    paradigm2Top[p]=topMax
top2Paradigms

In [ ]:
tiragesReduits["newRegex"]=tiragesReduits["regex"].apply(lambda x: paradigm2Top[x])

In [ ]:
tiragesReduits["Freq"]=tiragesReduits[marksN].fillna(0).apply(lambda x: sum(x),axis=1)

In [ ]:
for m in newDistMarks:
    tiragesReduits[m]=tiragesReduits[m].fillna(0)+newDistMarks[m]

In [ ]:
tiragesReduits

## Compter les effectifs des CF restantes

In [ ]:
newParadigmsCount={}
for t in top2Paradigms:
    newParadigmsCount[t]=sum([c for r,c in top2Paradigms[t]])
dfNewParadigms=pd.DataFrame.from_dict(newParadigmsCount,orient="index")
dfNewParadigms.columns=dfNewParadigms.columns.astype(str)
dfNewParadigms.columns=["effectif"]
dfNewParadigms.sort_values("effectif",ascending=False)

In [ ]:
newLexs=tiragesReduits.index.tolist()
newDistLexs=[v for k,v in probElements(tiragesReduits["Freq"].to_dict()).iteritems()]
#newDistLexs

In [ ]:
newLexTirs={}
newListeTirs=np.random.choice(newLexs,nbTokens,replace=True,p=newDistLexs)
for t in newListeTirs:
    if not t in newLexTirs:
        newLexTirs[t]=0
    newLexTirs[t]+=1

## Faire un nouveau tirage

=> mettre à jour les lexèmes et les classes flexionnelles

In [ ]:
newTirages=pd.concat([lexemes, tiragesColonnes], axis=1, sort=False)
for l in newLexTirs:
    lMarksFreq=probElements(tiragesReduits.loc[l,marksN].to_dict())
    lDistMarks=[lMarksFreq[m] for m in sorted(lMarksFreq.keys())]
    
    newCellTirs={}
    listeCases=np.random.choice(marksN,newLexTirs[l],replace=True,p=lDistMarks)
    for c in listeCases:
        if not c in newCellTirs:
            newCellTirs[c]=0
        newCellTirs[c]+=1
    if len(newCellTirs)==8 and debug:
        print l,tirages.loc[l,"CF"], newCellTirs
    for c in newCellTirs:
        newTirages.loc[l,c]=newCellTirs[c]


In [ ]:
newTirages

# Nouvelle architecture

1. initialiser le lexique
 - définir les cases, marques
    - nombre de cases
    - nombre de marques pour chaque case
 - définir le lexique-corpus
    - nombre de lexèmes
    - nombre de tokens
 - faire le tirage aléatoire
    - distribution des lexèmes
    - distribution des cases
    - distribution des marques
1. classer le lexique
 - emboîtement des classes flexionnelles
 - classes flexionnelles maximales
    - effectifs
 - rassemblement des paradigmes partiels
   - distribution dans les classes flexionnelles maximales
     - dans la classes maximale la plus fréquente
     - statistiquement entre les classes maximales 

# Initialiser le lexique


In [1]:
import pandas as pd
import numpy as np
import itertools as it
import random as rd
import networkx as nx
from matplotlib import pyplot as plt
import re,pickle
from IPython.display import display
debug=False

In [2]:
%matplotlib inline

## Choix des paramètres

In [3]:
nbCases=32
maxNbExposants=3
nbLexemes=10*1000
nbTokens=1000*1000*1000
epoques=25
initLexDist="c1.9"
initMarkDist="flat"
initCaseDist="c0.5"
epsilonFreq=0.1
lexProfileAleatoire=False
nbExposantsUniforme=True
sepPar="-"

### Fréquences et distributions

In [4]:
def distElements(elements,dist):
    nbElements=len(elements)
    if dist=="zipf":
        result=[float(nbElements)/(n+1) for n in range(nbElements)]
    elif dist=="flat":
        result=[1 for n in range(nbElements)]
    elif dist=="geo":
        result=[float(nbElements)/(n+1)**2 for n in range(nbElements)]
    elif dist.startswith("c"):
        coef=float(dist[1:])
        result=[float(nbElements)/(n+1)**coef for n in range(nbElements)]
    return result

def freq2prob(freq):
    totalFreq=sum(freq)
    result=[float(f)/totalFreq for f in freq]
    return result


In [5]:
distElements(range(10000),"c1.8")[-10:]

[0.0006319807847581873,
 0.0006318669416964166,
 0.0006317531305317552,
 0.0006316393512520754,
 0.0006315256038452553,
 0.0006314118882991789,
 0.000631298204601736,
 0.0006311845527408221,
 0.0006310709327043387,
 0.000630957344480193]

In [6]:
def initialiserMarques(nbCases):
    if nbExposantsUniforme:
        listeMarques=[maxNbExposants for i in range(nbCases)]
    else:
        listeMarques=[0]
        while max(listeMarques)<maxNbExposants:
            listeMarques=list(np.random.choice(range(1,maxNbExposants+1),nbCases))
    return listeMarques
    
def initialiserParadigme(nbCases,nbMarques=3,listeMarques=[]):
    if listeMarques==[]:
        listeMarques=[nbMarques for i in range(nbCases)]
    paradigme={i:listeMarques[i] for i in range(nbCases)}
    return paradigme

def initialiserListeDistMarques(listeMarques,dist=initMarkDist,distMarques=[]):
    if distMarques==[]:
        distMarques=[dist for i in listeMarques]
    result={i:distMarques[i] if i < len(distMarques) else dist for i in listeMarques}
    return result

def initialiserCasesDistMarques(nbMarkCase,listeDistMarques):
    result={}
    for i in nbMarkCase:
        result[i]=freq2prob(distElements(range(nbMarkCase[i]),dist=listeDistMarques[i]))
    return result

In [7]:
lexs=range(nbLexemes)
listeMarques=initialiserMarques(nbCases)
nbMarkCase=initialiserParadigme(nbCases,listeMarques=listeMarques)
listeDistMarques=initialiserListeDistMarques(nbMarkCase,initMarkDist,distMarques=[])
#casesDistMarques=initialiserCasesDistMarques(nbMarkCase,listeDistMarques)
probCasesMarques=initialiserCasesDistMarques(nbMarkCase,listeDistMarques)

In [8]:
nbPotentielClasses=reduce(lambda x, y: x * y, listeMarques)
print nbPotentielClasses,"classes flexionnelles potentielles"

1853020188851841 classes flexionnelles potentielles


In [9]:
probCasesMarques#=newProbCasesMarques
nbMarkCase

{0: 3,
 1: 3,
 2: 3,
 3: 3,
 4: 3,
 5: 3,
 6: 3,
 7: 3,
 8: 3,
 9: 3,
 10: 3,
 11: 3,
 12: 3,
 13: 3,
 14: 3,
 15: 3,
 16: 3,
 17: 3,
 18: 3,
 19: 3,
 20: 3,
 21: 3,
 22: 3,
 23: 3,
 24: 3,
 25: 3,
 26: 3,
 27: 3,
 28: 3,
 29: 3,
 30: 3,
 31: 3}

In [10]:
lexProbs=freq2prob(distElements(lexs,initLexDist))

In [11]:
marks=["c%02d"%i for i in range(nbCases)]
marksN=["n%02d"%i for i in range(nbCases)]
dfColumns=marks+marksN+["paradigme","CF"]

tirages=pd.DataFrame("", index=np.arange(len(lexs)), columns=dfColumns)
tirages[marksN]=0

In [12]:
distCases=freq2prob(distElements(marks,initCaseDist))

In [13]:
def tirerLexemes(lexProbs):
    lexTirs={}
    listeTirs=np.random.choice(lexs,nbTokens,replace=True,p=lexProbs)
    for t in listeTirs:
        if not t in lexTirs:
            lexTirs[t]=0
        lexTirs[t]+=1
    return lexTirs

In [14]:
def calculerChampParadigme(x):
    result=[]
    for c in marks:
        caseCount=c.replace("c","n")
#        print c,x[c],caseCount,x[caseCount]
        if x[c] in range(maxNbExposants):
            result.append(str(x[c]))
        else:
            result.append(".")
    return sepPar.join(result)

In [15]:
def getLDistCases(paradigme):
    lDistCases=[]
    lMarks=paradigme.split(sepPar)
    for n,m in enumerate(lMarks):
        if m==".":
            lDistCases.append(0)
        else:
            lDistCases.append(distCases[n])
    return freq2prob(lDistCases)

In [16]:
def getTirages(lexProbs,distCases):
    lexTirs=tirerLexemes(lexProbs)

    for l in lexTirs:
        if l%1000==0:
            print ".",
        cellTirs={}
        cellValue={}
        if lexProfileAleatoire:
            distCases=freq2prob(list(np.random.randint(1,101,size=nbCases)))
        if tirages.loc[l,"CF"] and "." in tirages.loc[l,"CF"]:
            lDistCases=getLDistCases(tirages.loc[l,"CF"])
        else:
            lDistCases=distCases
        listeCases=np.random.choice(marksN,lexTirs[l],replace=True,p=lDistCases)
        for c in listeCases:
            numCase=int(c[1:])
            v=c.replace("n","c")
            if not c in cellTirs:
                cellTirs[c]=0
            cellTirs[c]+=1
            if not v in cellValue:
                if tirages.loc[l,"CF"]:
                    cellValue[v]=int(tirages.loc[l,"CF"].split(sepPar)[numCase])
                else:
                    cellValue[v]=np.random.choice(range(nbMarkCase[numCase]),p=probCasesMarques[numCase])
        for c in cellTirs:
            tirages.loc[l,c]=cellTirs[c]
        for c in cellValue:
            tirages.loc[l,c]=cellValue[c]
    tirages["paradigme"]=tirages.apply(calculerChampParadigme,axis=1)

In [17]:
def matchParadigmes(p1,p2):
    chunks1=p1.split(sepPar)
    chunks2=p2.split(sepPar)
    return all([p1[i]==p2[i] or p1[i]=="." for i in range(len(p1))])

In [18]:
getTirages(lexProbs,distCases)

. . . . . . . . . .


In [19]:
def getParadimCountsGroups(tirages):
    paradigmeCounts=tirages.groupby(["paradigme"]).count()
    dictParadigmsCounts=paradigmeCounts["CF"].sort_values().to_dict()
    paradigmsGroups=tirages.groupby(["paradigme"]).groups.keys()
    print "nombre de paradigmes différents",len(dictParadigmsCounts)
    return dictParadigmsCounts,paradigmsGroups

In [20]:
#dictParadigmsCounts

In [21]:
def getParadigmMappings(paradigmsGroups):
    paradigmMappings={}
    for p in paradigmsGroups:
        for p2 in paradigmsGroups:
            m=matchParadigmes(p,p2)
            if m:
                if not p in paradigmMappings:
                    paradigmMappings[p]=set()
                paradigmMappings[p].add(p2)
    return paradigmMappings

In [22]:
dictParadigmsCounts,paradigmsGroups=getParadimCountsGroups(tirages)
paradigmMappings=getParadigmMappings(paradigmsGroups)

nombre de paradigmes différents 10000


In [23]:
def getParadigmTops(paradigmMappings,dictParadigmsCounts):
    paradigmTops={}
    for p in paradigmMappings:
        if len(paradigmMappings[p])==1:
            paradigmTops[p]=dictParadigmsCounts[p]
    return paradigmTops

In [24]:
paradigmTops=getParadigmTops(paradigmMappings,dictParadigmsCounts)

In [25]:
import operator
def getParadigm2Top(paradigmMappings,paradigmTops,dictParadigmsCounts):
    paradigm2Top={}
    top2Paradigms={}
    for p in paradigmMappings:
        lTops={m:paradigmTops[m] for m in paradigmMappings[p] if m in paradigmTops}
        topMax=max(lTops.iteritems(), key=operator.itemgetter(1))[0]
        if topMax not in top2Paradigms:
            top2Paradigms[topMax]=[]
        top2Paradigms[topMax].append((p,dictParadigmsCounts[p]))
        paradigm2Top[p]=topMax
    return top2Paradigms,paradigm2Top

In [26]:
top2Paradigms,paradigm2Top=getParadigm2Top(paradigmMappings,paradigmTops,dictParadigmsCounts)

In [27]:
def updateTirages():
    tirages["CF"]=tirages["paradigme"].apply(lambda x: paradigm2Top[x])
#    tirages["freq"]=tirages[marksN].replace({"":0}).apply(lambda x: sum(x),axis=1)
    tirages["freq"]=tirages[marksN].apply(lambda x: sum(x),axis=1)

In [28]:
updateTirages()

In [29]:
def updateProbCases():
    newProbCasesMarques={}
    for c in marks:
        lExpCount=tirages.groupby(c).count()["paradigme"]
        lNbExp=max([i for i in lExpCount.index.tolist() if i!=""])+1
        lExpFreq=[0 for f in range(lNbExp)]
        for k,v in tirages.groupby(c).count()["paradigme"].iteritems():
            if k!='':
                lExpFreq[k]=v
        if c not in newProbCasesMarques:
            newProbCasesMarques[c]=freq2prob(lExpFreq)
    return newProbCasesMarques

In [30]:
def updateLexProbs():
    newLexProbs=freq2prob([f+epsilonFreq for f in tirages["freq"].tolist()])
    return newLexProbs

In [31]:
def updateDistCases():
    newDistCases=freq2prob(tirages[marksN].sum().tolist())
    return newDistCases

In [32]:
probCasesMarques=updateProbCases()
lexProbs=updateLexProbs()
distCases=updateDistCases()

In [33]:
def archiveTirages(historique=[]):
    historique.append(tirages.copy())
    tirages[marks+["paradigme"]]=""
    tirages[marksN]=0
    return historique

In [34]:
historique=archiveTirages()

In [ ]:
tirages

,c00,c01,c02,c03,c04,c05,c06,c07,c08,c09,...,n25,n26,n27,n28,n29,n30,n31,paradigme,CF,freq
0,,,,,,,,,,,...,0,0,0,0,0,0,0,,1-2-0-1-1-0-2-0-0-0-2-1-0-1-2-1-0-2-0-1-0-2-0-...,571601472
1,,,,,,,,,,,...,0,0,0,0,0,0,0,,0-2-1-2-1-2-1-1-0-2-1-0-1-0-2-1-0-1-0-2-0-0-0-...,153159396
2,,,,,,,,,,,...,0,0,0,0,0,0,0,,2-2-0-1-2-2-1-0-0-0-1-1-0-0-2-1-0-0-2-2-0-2-1-...,70883986
3,,,,,,,,,,,...,0,0,0,0,0,0,0,,1-2-1-2-2-0-1-0-1-1-0-2-1-0-1-1-1-2-0-0-1-2-1-...,41047246
4,,,,,,,,,,,...,0,0,0,0,0,0,0,,1-1-2-0-2-2-0-1-1-0-0-0-0-1-1-1-2-2-1-0-0-2-0-...,26851887
5,,,,,,,,,,,...,0,0,0,0,0,0,0,,0-1-1-1-0-1-1-1-2-0-0-0-2-1-2-2-2-1-1-0-1-1-0-...,18994121
6,,,,,,,,,,,...,0,0,0,0,0,0,0,,1-0-2-1-0-0-0-1-0-0-0-0-0-0-0-1-0-0-2-0-1-2-2-...,14170610
7,,,,,,,,,,,...,0,0,0,0,0,0,0,,2-0-2-1-2-1-2-2-2-1-2-2-0-0-0-1-1-2-2-1-0-2-2-...,10990670
8,,,,,,,,,,,...,0,0,0,0,0,0,0,,1-2-1-1-2-2-1-2-2-1-2-1-0-0-0-2-1-1-1-0-1-0-1-...,8789831
9,,,,,,,,,,,...,0,0,0,0,0,0,0,,2-1-2-0-1-2-2-2-2-0-0-2-1-0-2-2-0-2-1-2-0-2-0-...,7194815


In [ ]:
for i in range(epoques):
    getTirages(lexProbs,distCases)

    dictParadigmsCounts,paradigmsGroups=getParadimCountsGroups(tirages)
    paradigmMappings=getParadigmMappings(paradigmsGroups)

    paradigmTops=getParadigmTops(paradigmMappings,dictParadigmsCounts)
    top2Paradigms,paradigm2Top=getParadigm2Top(paradigmMappings,paradigmTops,dictParadigmsCounts)

    updateTirages()

    probCasesMarques=updateProbCases()
    lexProbs=updateLexProbs()
    distCases=updateDistCases()

    historique=archiveTirages(historique)

. . . . . . . . . . nombre de paradigmes différents 10000
. . . . . . . . . . nombre de paradigmes différents 9999
. . . . . . . . . . nombre de paradigmes différents

In [ ]:
for n,h in enumerate(historique):
    hLex=h[h["freq"]>=1]
    df=hLex.groupby("CF").count()[["paradigme"]].sort_values("paradigme")
    df["freq"]=hLex.groupby("CF").sum()[["freq"]]
    df["maxF"]=hLex.groupby("CF").max()[["freq"]]
    df["minF"]=hLex.groupby("CF").min()[["freq"]]
    df["medF"]=hLex.groupby("CF").median()[["freq"]]
    df["meanF"]=hLex.groupby("CF").mean()[["freq"]]
    print n,len(df)
    display (df)
    print
    print

In [ ]:
with open("historique-C%02d-E%02d-%s.pkl"%(nbCases,maxNbExposants,(initLexDist[0]+initCaseDist[0]+initMarkDist[0]).upper()),"wb") as outFile:
    pickle.dump(historique, outFile, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
h=historique[-1]
hLex=h[h["freq"]>=1]
df=hLex.groupby("CF").count()[["paradigme"]].sort_values("paradigme")
df["freq"]=hLex.groupby("CF").sum()[["freq"]]
df["maxF"]=hLex.groupby("CF").max()[["freq"]]
df["minF"]=hLex.groupby("CF").min()[["freq"]]
df["medF"]=hLex.groupby("CF").median()[["freq"]]
df["meanF"]=hLex.groupby("CF").mean()[["freq"]]

In [ ]:
df[df["paradigme"]==1].sort_values("meanF")

In [ ]:
df["paradigme"].sum()

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist

In [ ]:
df.index.sort_values("paradigme",ascending=False)[0].tolist()

In [ ]:
def categoricalExponents(cf):
    cfMarks=cf.split("-")
    result=[]
    for k,v in nbMarkCase.iteritems():
        for i in range(v):
            if cfMarks[k]=="." or int(cfMarks[k])==i:
                result.append(1)
            else:
                result.append(0)
    return result

In [ ]:
#categoricalExponents(".-2-2-2-2-2-2-2")

In [ ]:
classesFlexionnelles=[]
labelCF=df.index.sort_values("paradigme",ascending=False)[0].tolist()
for cf in labelCF:
    classesFlexionnelles.append(categoricalExponents(cf))

In [ ]:
#classesFlexionnelles,labelCF

In [ ]:
Z = linkage(classesFlexionnelles)

In [ ]:
# calculate full dendrogram
plt.figure(figsize=(250, 10))
plt.title('Hierarchical Clustering Dendrogram')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    leaf_rotation=90.,  # rotates the x axis labels
    leaf_font_size=8.,  # font size for the x axis labels
    labels=labelCF
)
plt.show()



In [ ]:
plt.figure(figsize=(25, 10))
plt.title('Hierarchical Clustering Dendrogram (truncated)')
plt.xlabel('sample index')
plt.ylabel('distance')
dendrogram(
    Z,
    truncate_mode='lastp',  # show only the last p merged clusters
    p=12,  # show only the last p merged clusters
    show_leaf_counts=False,  # otherwise numbers in brackets are counts
    leaf_rotation=90.,
    leaf_font_size=12.,
    show_contracted=True,  # to get a distribution impression in truncated branches
    labels=labelCF
)
plt.show()

In [ ]:
with open("historique-C%02d-E%02d-%s.pkl"%(nbCases,maxNbExposants,(initLexDist[0]+initCaseDist[0]+initMarkDist[0]).upper()),"rb") as inFile:
    history=pickle.load(inFile)

In [ ]:
h=history[-1]
h=h[(h["freq"]>0)]
dfGroup=h.groupby("CF")
countCF=dfGroup.count()[["paradigme"]]

In [ ]:
ax=countCF[countCF["paradigme"]>1].sort_values("paradigme",ascending=False).plot()
plt.figure(figsize=(25, 10))
plt.title(u'Lexème par CF')
plt.xlabel('CF')
plt.ylabel(u'Nb de lexèmes')
